<a href="https://colab.research.google.com/github/PriyaMurali/Data-Analysis/blob/main/HiveMySQLDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from collections import Counter
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

In [2]:
!apt-get update
!apt-get install --fix-missing openjdk-8-jdk-headless -qq > /dev/null


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http

In [3]:
!pip install mysql-connector-python


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.4 MB 15.0 MB/s 


In [13]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [ ]:
#HIve Mysql Example
import mysql.connector
import pandas as pd

appName = "PySpark MySQL Example - via mysql.connector"
master = "local"

spark = SparkSession.builder.master(master).appName(appName).getOrCreate()

# Establish a connection
conn = mysql.connector.connect(user='***', database='***',
                               password='***',
                               host="localhost",
                               port=10101)
cursor = conn.cursor()
query = "SELECT id, value FROM test_table"
# Create a pandas dataframe
pdf = pd.read_sql(query, con=conn)
conn.close()

# Convert Pandas dataframe to spark DataFrame
df = spark.createDataFrame(pdf)

df.show()

In [8]:
from pyspark.sql import HiveContext
hc = HiveContext(spark.sparkContext)
print(hc)


/content/spark-3.3.0-bin-hadoop3/python/pyspark/sql/context.py:721: FutureWarning: HiveContext is deprecated in Spark 2.0.0. Please use SparkSession.builder.enableHiveSupport().getOrCreate() instead.
  FutureWarning,
/content/spark-3.3.0-bin-hadoop3/python/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [11]:
hc.sql('create database db') #where db is the database name
hc.sql('use db')

DataFrame[]

In [17]:
##Storing Data in HiveContext
from pyspark.sql.types import StructType, StructField, FloatType, StringType
iris_schema = StructType([
StructField("Sepal_Length", FloatType(), True),
StructField("Sepal_Width", FloatType(), True),
StructField("Petal_Length", FloatType(), True),
StructField("Petal_Width", FloatType(), True),
StructField("Species", StringType(), True)
])
iris1_df1 = spark.read.csv(path='/content/iris.csv',sep=',',header=True,schema=iris_schema)
hc.sql('use db')
from pyspark.sql import DataFrameWriter
dfw = DataFrameWriter(iris1_df1)
dfw.saveAsTable(name="iris",mode='overwrite')


In [18]:
hc.tableNames()

['iris']

In [24]:
#Retriving Data from HiveContext
hc.sql('Select count(*) as RecordCount from iris').show()

+-----------+
|RecordCount|
+-----------+
|        150|
+-----------+

